In [1]:
import pytube
from pytube import YouTube
from pytube.exceptions import VideoUnavailable
from pytube.exceptions import VideoPrivate, AgeRestrictedError,LiveStreamError, RecordingUnavailable, MembersOnly, VideoRegionBlocked
import pandas as pd
import numpy as np
import os
from moviepy.editor import *
import pytube.exceptions

In [2]:
path = '../raw_data/train_reduced.csv'

In [3]:
df = pd.read_csv(path)

In [4]:
df.shape

(77000, 5)

In [5]:
labels = df['label'].unique()

In [7]:
julian_label = labels[:175]

In [8]:
julian_label

array(['clay pottery making', 'news anchoring', 'using bagging machine',
       'javelin throw', 'climbing a rope', 'sipping cup',
       'flipping pancake', 'tickling', 'watering plants',
       'eating spaghetti', 'dribbling basketball', 'calligraphy',
       'brushing floor', 'chiseling stone', 'crossing eyes',
       'mountain climber (exercise)', 'using inhaler', 'slapping',
       'shaking head', 'driving tractor', 'delivering mail',
       'riding a bike', 'playing didgeridoo', 'putting in contact lenses',
       'doing jigsaw puzzle', 'playing with trains', 'brushing teeth',
       'washing hair', 'opening bottle (not wine)', 'finger snapping',
       'making balloon shapes', 'pouring beer', 'feeding birds',
       'wading through water', 'petting cat', 'playing drums',
       'playing ukulele', 'throwing knife', 'knitting', 'dining',
       'hurdling', 'throwing water balloon', 'tiptoeing',
       'changing gear in car', 'playing cymbals',
       'making paper aeroplanes', 'va

In [11]:
frames = []

for i in julian_label:
    g = df[df.label == i]
    frames.append(g)

julian_df = pd.concat(frames)

In [12]:
julian_df.shape

(19250, 5)

In [14]:
julian_df.head(20)

,label,youtube_id,time_start,time_end,split
0,clay pottery making,LHtsiogBKvM,34,44,train
1,clay pottery making,K4O9S_gpgbY,64,74,train
2,clay pottery making,kVAScTNRnwA,72,82,train
3,clay pottery making,RE9bEWMzVtg,0,10,train
4,clay pottery making,4mbdJHOnPuA,1,11,train
5,clay pottery making,P8styuac15I,44,54,train
6,clay pottery making,-GU00Yxwafc,29,39,train
7,clay pottery making,EOxV6ZCjy8U,29,39,train
8,clay pottery making,FP0XDogwwW8,22,32,train
9,clay pottery making,6Yjq8VVwmkc,18,28,train


In [18]:
def remove_long_videos(df):
    test = df.copy()
    #count is just for testing
    count = 0
    #list of all the lengths in order to build the new column
    lengths = []
    #iterating over through all the videos
    for video in range(0,len(test)):
        url = "https://www.youtube.com/watch?v="+test.youtube_id[video]
        yt = YouTube(url)
        #handling error messages
        try:
            stream = yt.streams
        except (KeyError,VideoPrivate,VideoUnavailable,AgeRestrictedError,LiveStreamError, RecordingUnavailable, MembersOnly, VideoRegionBlocked):
            test.drop(video,inplace=True)
            continue
        #drop video if longer than 5min
        if yt.length > 90:
            test.drop(video,inplace=True)
        #if video is already clipped, but end_time is higher than lenght of the video, we set new start and end points
        elif yt.length == 10:
            test.time_start[video] = 0
            test.time_end[video] = yt.length
            length = yt.length
            lengths.append(length)
        #if time_end greater than the length and it's not clipped we drop it
        elif yt.length < test.time_end[video]:
            test.drop(video,inplace=True)
        #appending the lengths to the list     
        else:
            length = yt.length
            lengths.append(length)
    #creating the new column    
    test["lenghts"] = lengths
    #returning the new dataframe
    return test.reset_index(drop=True)   